In [2]:
import numpy as np
import pandas as pd

In [3]:
def calcular_sindrome(patron_error, matriz_Ht):
    sindrome = []
    detectable = []
    for i in patron_error:
        valor = []
        i2 = [np.float64(j) for j in np.array(list(i))]
        for x in range(len(matriz_Ht[0])):
            multiply_ih = i2*matriz_Ht[:,x]
            valid = np.count_nonzero(multiply_ih == 1)
            if valid%2 != 0:
                valor.append(1)
            else:
                valor.append(0)
        sindrome.append(valor)
        if valor == [0, 0, 0]:
            detectable.append("no")
        else:
            detectable.append("si")
    return sindrome, detectable
    

In [170]:
n=7
k=4
p_e = 1e-3
matriz_P = np.array([[1,1,0],[0,1,1],[1,1,1],[1,0,1]], dtype=int)
matriz_Ik = np.identity(k,dtype=int)
matriz_G = np.concatenate((matriz_P.T,matriz_Ik.T),axis=0,dtype=int).T
matriz_H = np.concatenate((matriz_Ik[:-1].T[:-1],matriz_P),axis=0,dtype=int).T
matriz_Ht = matriz_H.T

In [172]:
## calculo de patrones de error
patron_error = np.array(["{0:07b}".format(x) for x in range(128)])
cantidad_error = np.array([x.count('1') for x in patron_error])
sindrome, detectable = calcular_sindrome(patron_error,matriz_Ht)
corregibles = list(map(lambda x: "no" if x>1 else "si", cantidad_error))
probabilidad = [p_e**x * (1-p_e)**(n-x) for x in cantidad_error]

In [173]:
calculo_patrones = pd.DataFrame({'patron error':patron_error,'cantidad error':cantidad_error,'sindrome':sindrome,'detectable':detectable,'corregibles':corregibles,'probabilidad':probabilidad})
calculo_patrones

,patron error,cantidad error,sindrome,detectable,corregibles,probabilidad
0,0000000,0,"[0, 0, 0]",no,si,9.930210e-01
1,0000001,1,"[1, 0, 1]",si,si,9.940150e-04
2,0000010,1,"[1, 1, 1]",si,si,9.940150e-04
3,0000011,2,"[0, 1, 0]",si,no,9.950100e-07
4,0000100,1,"[0, 1, 1]",si,si,9.940150e-04
...,...,...,...,...,...,...
123,1111011,6,"[0, 1, 1]",si,no,9.990000e-19
124,1111100,5,"[0, 1, 0]",si,no,9.980010e-16
125,1111101,6,"[1, 1, 1]",si,no,9.990000e-19
126,1111110,6,"[1, 0, 1]",si,no,9.990000e-19


In [226]:
###################### Transmisor ###################
f = open ('mensaje_codificado.txt','r')
mensaje = f.read()
while(len(mensaje)%4 != 0):
    mensaje+='0'
mensaje = np.array(list(map(list, zip(*[map(int, list(mensaje))] * 4))),dtype=int)
m2 = mensaje.copy()
mensaje = mensaje @ matriz_G
mensaje_generadora = [[0 if a%2==0 else 1 for a in x] for x in mensaje]
mensaje_enviar = "".join(["".join([str(ele) for ele in sub]) for sub in mensaje_generadora])
mensaje_enviar

'111001010100011010001000000011001011110010001011100011010001101001011111010000001101001011101101001010001010001100011011111111010001111001011101000011010010100010111001101110000000000001101110010101110011000110100101111010001010001100011001110010111001001011100000000001101011100111111111100101000000001101001100101100011011100100100011110010101011100100011101110010111000111001001011110001101011100010001111001011100101011100110100010100011000000000101111101000110100011010001110010000000000000001101000011010000000000111001001011111001010000000100011011001010000000101110001000110000000111001001101001110010101000100011011001011000000011100101100101110100011010000111001110010101000111110010001011101101001010001000000010100010011010100011011001011011100001101000000001010001011010000000001000110110010110100010000000101000101000110000000000110100110100010111110100010100010100011101110010001100110100001011111010000001101000000010001100000000000000001101001101000000110101000111101000011010001101

In [229]:
############### Receptor ##################


[0 0 0 1]
[1 0 1 0 0 0 1]


[1, 0, 1, 0, 0, 0, 1]